In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

 * sqlite:///chinook.db
Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
with usa_tracks_sold as
    (
    SELECT il.*
    FROM invoice_line as il
    INNER JOIN invoice as i ON i.invoice_id = il.invoice_id
    INNER JOIN customer as c ON c.customer_id = i.customer_id
    WHERE c.country = 'USA'
    )
    
SELECT g.name as genre,
    COUNT(uts.invoice_line_id) as tracks_sold,
    ROUND(CAST(COUNT(uts.invoice_line_id) AS FLOAT) / (
                                                SELECT COUNT(*) FROM usa_tracks_sold), 3) as percentage_sold
    FROM usa_tracks_sold as uts
    INNER JOIN track as t ON t.track_id = uts.track_id
    INNER JOIN genre as g ON g.genre_id = t.genre_id
    GROUP BY 1
    ORDER BY 2 DESC
    LIMIT 3;

 * sqlite:///chinook.db
Done.


genre,tracks_sold,percentage_sold
Rock,561,0.534
Alternative & Punk,130,0.124
Metal,124,0.118


In [4]:
%%sql

WITH customer_support_rep_sales AS
    (
    SELECT c.customer_id,
    c.support_rep_id,
    SUM(i.total) as total_sales
    FROM customer as c
    INNER JOIN invoice as i ON i.customer_id = c.customer_id
    GROUP BY 1, 2)
    
SELECT e.first_name || " " || e.last_name as employee_name,
    e.hire_date,
    SUM(csp.total_sales) 
    FROM employee as e
    INNER JOIN customer_support_rep_sales as csp ON csp.support_rep_id = e.employee_id
    GROUP BY 1;

 * sqlite:///chinook.db
Done.


employee_name,hire_date,SUM(csp.total_sales)
Jane Peacock,2017-04-01 00:00:00,1731.5099999999998
Margaret Park,2017-05-03 00:00:00,1584.0000000000002
Steve Johnson,2017-10-17 00:00:00,1393.92


In [5]:
%%sql
WITH country_or_other AS
(
SELECT 
    CASE 
        WHEN (
            SELECT COUNT(*)
            FROM customer
            where country = c.country) = 1 then 'Other'
        ELSE c.country
    END AS country,
    c.customer_id,
    il.*
    FROM invoice_line as il
    INNER JOIN invoice as i ON i.invoice_id = il.invoice_id
    INNER JOIN customer as c ON c.customer_id = i.customer_id
)

SELECT 
    country,
    customers,
    total_sales,
    average_order,
    customer_lifetime_value
    
FROM 
(SELECT
     country,
    COUNT(DISTINCT(customer_id)) as customers,
    SUM(unit_price) as total_sales,
    SUM(unit_price) / COUNT(DISTINCT(customer_id)) as customer_lifetime_value,
    SUM(unit_price) / COUNT(DISTINCT(invoice_id)) as average_order,
    CASE
     WHEN
         country = 'Other' THEN 1
     ELSE 0
     END as sort
FROM country_or_other
    GROUP BY country
    ORDER BY sort ASC, total_sales DESC);
        

 * sqlite:///chinook.db
Done.


country,customers,total_sales,average_order,customer_lifetime_value
USA,13,1040.490000000008,7.942671755725252,80.03769230769292
Canada,8,535.5900000000034,7.047236842105309,66.94875000000043
Brazil,5,427.68000000000245,7.011147540983647,85.53600000000048
France,5,389.0700000000021,7.781400000000042,77.81400000000042
Germany,4,334.6200000000016,8.161463414634186,83.6550000000004
Czech Republic,2,273.24000000000103,9.108000000000034,136.62000000000052
United Kingdom,3,245.52000000000078,8.768571428571457,81.84000000000026
Portugal,2,185.13000000000022,6.3837931034482835,92.56500000000011
India,2,183.1500000000002,8.72142857142858,91.5750000000001
Other,15,1094.9400000000085,7.448571428571486,72.99600000000056


In [21]:
%%sql

WITH invoice_first_track AS
(
SELECT il.invoice_id,
    il.track_id as first_track_id
    FROM invoice_line as il
GROUP BY 1
)

SELECT Album_purchased,
    COUNT(invoice_id) as number_of_invoices,
    ROUND((CAST(COUNT(invoice_id) AS FLOAT) / (SELECT COUNT(*) FROM invoice))*100, 2) as percent
    
    FROM 
        (
        SELECT ifs.*,
        CASE
           WHEN ( 
                SELECT t.track_id 
                    FROM track as t
                    where t.album_id = (
                                        SELECT t2.album_id
                                            FROM track as t2
                                            WHERE t2.track_id = ifs.first_track_id
                                        )
                EXCEPT

                SELECT il2.track_id
                    FROM invoice_line as il2
                    WHERE il2.invoice_id = ifs.invoice_id
               ) is null
            and
                (
                SELECT il2.track_id
                FROM invoice_line as il2
                WHERE il2.invoice_id = ifs.invoice_id

                EXCEPT
                    
                SELECT t.track_id 
                FROM track as t
                where t.album_id = (
                                    SELECT t2.album_id
                                        FROM track as t2
                                        WHERE t2.track_id = ifs.first_track_id
                                    )
                ) is null
            THEN 'YES'
            ELSE 'NO'
            END AS 'Album_purchased'
    FROM invoice_first_track as ifs
        )
    GROUP BY Album_purchased;
    

 * sqlite:///chinook.db
Done.


Album_purchased,number_of_invoices,percent
NO,500,81.43
YES,114,18.57
